In [1]:
!pip install -q pandas numpy sentence-transformers faiss-cpu transformers accelerate


You should consider upgrading via the 'C:\Users\user\Documents\NLPP\coba2\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [41]:
import pandas as pd
import numpy as np
import faiss

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [46]:
kb = pd.read_csv("knowledge_base_haid.csv", encoding='latin1')
kb.head()

,id,category,question,answer
0,1,Pengertian,Apa itu haid?,Haid atau menstruasi adalah proses keluarnya d...
1,2,Pengertian,Apa penyebab terjadinya haid?,Haid terjadi karena luruhnya lapisan dinding r...
2,3,Siklus,Berapa lama siklus haid normal?,Siklus haid normal berkisar antara 21 hingga 3...
3,4,Siklus,Berapa lama haid berlangsung?,Durasi haid normal berlangsung antara 3 hingga...
4,5,Gejala,Apa saja tanda-tanda akan haid?,"Tanda-tanda akan haid meliputi nyeri perut, ny..."


In [50]:
documents = (
    kb["question"].astype(str) + " " + kb["answer"].astype(str)
).tolist()

documents[:3]

['Apa itu haid? Haid atau menstruasi adalah proses keluarnya darah dari rahim melalui vagina yang terjadi secara alami setiap bulan sebagai bagian dari siklus reproduksi perempuan.',
 'Apa penyebab terjadinya haid? Haid terjadi karena luruhnya lapisan dinding rahim akibat tidak terjadinya pembuahan sel telur.',
 'Berapa lama siklus haid normal? Siklus haid normal berkisar antara 21 hingga 35 hari, dengan rata-rata 28 hari.']

In [51]:
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


In [52]:
embeddings = embed_model.encode(
    documents,
    show_progress_bar=True
)

embeddings.shape


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

(50, 384)

In [53]:
dim = embeddings.shape[1]

index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

print("Jumlah dokumen dalam index:", index.ntotal)


Jumlah dokumen dalam index: 50


In [54]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=80,
    temperature=0.7
)


Device set to use cpu


In [55]:
def retrieve_context(query, k=3):
    query_emb = embed_model.encode([query])
    distances, indices = index.search(
        np.array(query_emb), k
    )
    contexts = [documents[i] for i in indices[0]]
    return "\n".join(contexts)


In [ ]:
def build_prompt(context, question):
    prompt = f"""
Kamu adalah chatbot kesehatan khusus membahas menstruasi (haid).
Jawablah dengan bahasa yang sopan, jelas, dan mudah dipahami.
Jika pertanyaan berpotensi medis serius, sarankan konsultasi tenaga medis.

Context:
{context}

Pertanyaan:
{question}

Jawaban:
"""
    return prompt


In [57]:
def chatbot_haid(question):
    context = retrieve_context(question)
    prompt = build_prompt(context, question)

    response = llm(prompt)[0]["generated_text"]
    return response.split("Jawaban:")[-1].strip()


In [58]:
print(chatbot_haid("jarak siklus haid itu berapa hari?"))


Jarak siklus haid biasanya berkisar antara 21-35 hari. Namun, ini bisa berubah sesuai dengan individu dan siklus sehari-hari mereka. Jadi, untuk mengukur jarak siklus haid Anda, cobalah untuk mencatat waktu haid Anda selama beberapa minggu dan merujuk pada data tersebut untuk mem


In [59]:
!pip install -q streamlit pyngrok pandas numpy faiss-cpu sentence-transformers transformers torch


In [60]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

st.set_page_config(
    page_title="Chatbot Edukasi Haid",
    page_icon="🌸",
    layout="centered"
)

st.markdown("""
<style>
.chat-box {background:#f9f9f9;padding:15px;border-radius:12px;margin-bottom:10px;}
.user {color:#1f77b4;font-weight:bold;}
.bot {color:#2ca02c;font-weight:bold;}
</style>
""", unsafe_allow_html=True)

st.title("🌸 Chatbot Edukasi Haid")
st.caption("Chatbot RAG + LLM untuk edukasi menstruasi")

@st.cache_resource
def load_data():
    kb = pd.read_csv("knowledge_base_haid_rag.csv")
    docs = (kb["question"] + " " + kb["answer"]).tolist()
    return docs

docs = load_data()

@st.cache_resource
def load_embedder():
    return SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

embedder = load_embedder()

@st.cache_resource
def build_index(docs):
    emb = embedder.encode(docs)
    index = faiss.IndexFlatL2(emb.shape[1])
    index.add(np.array(emb))
    return index

index = build_index(docs)

@st.cache_resource
def load_llm():
    model = "Qwen/Qwen2.5-0.5B-Instruct"
    tok = AutoTokenizer.from_pretrained(model)
    mdl = AutoModelForCausalLM.from_pretrained(model, device_map="auto")
    return pipeline(
        "text-generation",
        model=mdl,
        tokenizer=tok,
        max_new_tokens=80,
        temperature=0.5
    )

llm = load_llm()

def retrieve(q, k=3):
    qemb = embedder.encode([q])
    _, idx = index.search(np.array(qemb), k)
    return "\n".join([docs[i] for i in idx[0]])

def chat(q):
    ctx = retrieve(q)
    prompt = f"""
Kamu adalah chatbot edukasi haid.
Gunakan konteks berikut untuk menjawab singkat dan jelas.

Konteks:
{ctx}

Pertanyaan:
{q}

Jawaban:
"""
    out = llm(prompt)[0]["generated_text"]
    return out.split("Jawaban:")[-1].strip()

if "chat" not in st.session_state:
    st.session_state.chat = []

q = st.text_input("💬 Tanyakan tentang haid")

if st.button("Kirim") and q:
    a = chat(q)
    st.session_state.chat.append(("user", q))
    st.session_state.chat.append(("bot", a))

for r, m in st.session_state.chat:
    st.markdown(f"<div class='chat-box'><b>{'Kamu' if r=='user' else 'Bot'}:</b><br>{m}</div>", unsafe_allow_html=True)


Overwriting app.py


In [64]:
!streamlit run app.py &>/content/logs.txt &

from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("OPEN:", public_url)


OPEN: NgrokTunnel: "https://hexamerous-unimpairable-lucca.ngrok-free.dev" -> "http://localhost:8501"


In [63]:
from pyngrok import ngrok

# !!! IMPORTANT: Replace 'YOUR_AUTH_TOKEN' with your actual ngrok authtoken. !!!
# You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token('36pwrTMawezfLa2vaukTrLYkoeq_3bU8nqQGFJ7QdLkhDRnL2')

public_url = ngrok.connect(8501)
print("STREAMLIT URL:", public_url)

!streamlit run app.py &>/content/logs.txt &

STREAMLIT URL: NgrokTunnel: "https://hexamerous-unimpairable-lucca.ngrok-free.dev" -> "http://localhost:8501"
